## Exercise 1

Poisson regression is a Generalized Linear Model, used to model count data. It takes the form

$$\mathbb{E}(\mu|x)=\exp(w_1\,x_1+\ldots+w_k\,x_k+b),$$

where the observed counts $y$ are drawn from a Poisson distribution on the expected counts: 

$$y_i \sim \text{Poisson}(\mu_i).$$

1. Download and import Load the smoking dataset from: [https://data.princeton.edu/wws509/datasets/#smoking](https://data.princeton.edu/wws509/datasets/#smoking). Then perform a train-test split on the data;


In [111]:
%reset -f
import numpy as np
import pandas as pd

import pyro
import pyro.distributions as dist
import pyro.optim as optim

from pyro.infer import SVI, Trace_ELBO
from pyro.infer import Predictive

import torch
import torch.distributions.constraints as constraints

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

pyro.set_rng_seed(42)

In [53]:
data = pd.read_csv("https://data.princeton.edu/wws509/datasets/smoking.raw",sep="\t",header=None)
data.columns = ["Age_Group","Smoking_Status","Population","Deaths"]
data.head()

,Age_Group,Smoking_Status,Population,Deaths
0,1,1,656,18
1,2,1,359,22
2,3,1,249,19
3,4,1,632,55
4,5,1,1067,117


* age at the start of follow-up: in five-year age groups coded 1 to 9 for 40-44, 45-49, 50-54, 55-59, 60-64, 65-69, 70-74, 75-79, 80+.
* smoking status: coded 1 = never smoked, 2 = smoked cigars or pipe only, 3 = smoked cigarettes and cigar or pipe, and 4 = smoked cigarettes only,
* population: number of male pensioners followed, and
* deaths: number of deaths in a six-year period.

In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Age_Group       36 non-null     int64
 1   Smoking_Status  36 non-null     int64
 2   Population      36 non-null     int64
 3   Deaths          36 non-null     int64
dtypes: int64(4)
memory usage: 1.2 KB


Now we perform a train-test split on the data:
- **train data** - 80% of the observations will be used to perfom inference on our model
- **test data** - the remaining 20% will be used for testing the correctness of posterior predictions 

In [55]:
# scale the population since it dominates the predictors  * X
scaler = MinMaxScaler()
data["Population"] = scaler.fit_transform(pd.DataFrame(data["Population"]))

In [56]:
deaths = torch.tensor(data["Deaths"].values, dtype=torch.float)
predictors = torch.stack([torch.tensor(data[column].values,dtype=torch.float) 
                            for column in data.columns if column != "Deaths"],1)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(predictors, deaths, test_size=0.20, 
                                                    random_state=42,shuffle=True)

print("X_train.shape =", X_train.shape,"\ny_train.shape =", y_train.shape)
print("\nX_test.shape =", X_test.shape,"\ny_test.shape =", y_test.shape)

X_train.shape = torch.Size([28, 3]) 
y_train.shape = torch.Size([28])

X_test.shape = torch.Size([8, 3]) 
y_test.shape = torch.Size([8])


2. Fit a Poisson bayesian regression model using the number of deaths as the response variable and the other columns as the explanatory variables;

The target variable in our model is the number of deaths and we wish to infer the parameters corresponding to the following predictors

$$
\text{Deaths}=\text{exp}(w_0\cdot\text{Age_Group}+w_1\cdot\text{Smoking_Status}+w_2\cdot\text{Population}+b) + \epsilon
$$

We set a normal prior on $w$, a Log-Normal on the bias term $b$ and a uniformly distributed std for the gaussian noise on $\hat{y}$

\begin{align*}
w&\sim\mathcal{N}(0,1)\\
b&\sim\text{LogNormal}(0,1)\\
\hat{\mu}&= \text{exp}(w x+ b) \\
y &\sim \text{Poisson}(\hat{\mu}).
\end{align*}

Then we define the family of posterior distributions, by setting a Gamma distribution on $w$ and a Log-Normal on $b$, and run SVI inference on $(x,y)$ data.

Notice the prior distribution on the bias term makes this regression problem analytically intractable.

**will be read**

https://towardsdatascience.com/an-illustrated-guide-to-the-poisson-regression-model-50cccba15958

https://ncss-wpengine.netdna-ssl.com/wp-content/themes/ncss/pdf/Procedures/NCSS/Poisson_Regression.pdf

**Poisson ile ilgili okuma yapılacak, tanım aralıklarına bakılacak**

**w  için negatif değer bulunmaması gerekiyor ona bakılacak, muhati nasıl pozitif yaparım** 

In [58]:
pyro.clear_param_store()
def death_model(predictors, deaths):
    
    n_observations, n_predictors = predictors.shape
    
    # sample weights
    w = pyro.sample("w", dist.Normal(torch.zeros(n_predictors), 
                                        torch.ones(n_predictors)))
    b = pyro.sample("b", dist.LogNormal(torch.zeros(1), torch.ones(1)))
    
    mu_hat = torch.exp((w*predictors).sum(dim=1) + b)
    
    # condition on the observations
    with pyro.plate("deaths", len(deaths)):
        pyro.sample("obs", dist.Poisson(mu_hat), obs=deaths)
        
def death_guide(predictors, deaths=None):
    
    n_observations, n_predictors = predictors.shape
        
    w_loc = pyro.param("w_loc", torch.rand(n_predictors), constraint=constraints.positive)
    w_scale = pyro.param("w_scale", torch.rand(n_predictors), 
                         constraint=constraints.positive)
    
    w = pyro.sample("w", dist.Gamma(w_loc, w_scale))
    
    b_loc = pyro.param("b_loc", torch.rand(1))
    b_scale = pyro.param("b_scale", torch.rand(1), constraint=constraints.positive)
    
    b = pyro.sample("b", dist.LogNormal(b_loc, b_scale))
    


#smokeguide = pyro.infer.autoguide.AutoMultivariateNormal(death_model)
    
death_svi = SVI(model=death_model, guide=death_guide, 
              optim=optim.ClippedAdam({'lr' : 0.01}), 
              loss=Trace_ELBO())

for step in range(2000):
    loss = death_svi.step(X_train, y_train)/len(X_train)
    if step % 100 == 0:
        print(f"Step {step} : loss = {loss}")


Step 0 : loss = 70.29648329956191
Step 100 : loss = 147.51487459880966
Step 200 : loss = 762.3052494632346
Step 300 : loss = 1110.694842006479
Step 400 : loss = 129166.11681348724
Step 500 : loss = 27257.90517469389
Step 600 : loss = 945.941014362233
Step 700 : loss = 646.1101501371179
Step 800 : loss = 4804417.224761373
Step 900 : loss = 1449166.9001140913
Step 1000 : loss = 154.2531350031495
Step 1100 : loss = 646.3478335163423
Step 1200 : loss = 1969.7512530982494
Step 1300 : loss = 46173742.12855301
Step 1400 : loss = 400650.3777360767
Step 1500 : loss = 237.35848464603936
Step 1600 : loss = 673.6445791700056
Step 1700 : loss = 126.5137655287981
Step 1800 : loss = 6307899.792603889
Step 1900 : loss = 13187.547154643706


In [59]:
print("Inferred params:", list(pyro.get_param_store().keys()), end="\n\n")

# w_i and b posterior mean
inferred_w = pyro.get_param_store()["w_loc"]
inferred_b = pyro.get_param_store()["b_loc"]

for i,w in enumerate(inferred_w):
    print(f"w_{i} = {w.item():.8f}")
print(f"b = {inferred_b.item():.8f}")

Inferred params: ['w_loc', 'w_scale', 'b_loc', 'b_scale']

w_0 = 0.58638030
w_1 = 0.59420520
w_2 = 0.81937784
b = -0.05656544


**Posterior predictive distribution**

We can use the `Predictive` utility class, corresponding to the posterior predictive distribution, to evaluate our model on test data. Here we compute some summary statistics (mean, std and qualtiles) on $100$ samples from the posterior predictive:

In [60]:
# print latent params quantile information
def summary(samples):
    stats = {}
    for par_name, values in samples.items():
        marginal = pd.DataFrame(values)
        percentiles=[.05, 0.5, 0.95]
        describe = marginal.describe(percentiles).transpose()
        stats[par_name] = describe[["mean", "std", "5%", "50%", "95%"]]
    return stats

# define the posterior predictive
predictive = Predictive(model=death_model, guide=death_guide, num_samples=100,
                        return_sites=("w","b"))

# get posterior samples on test data
svi_samples = {k: v.detach().numpy() for k, v in predictive(X_test, y_test).items()}

# show summary statistics
for key, value in summary(svi_samples).items():
    print(f"Sampled parameter = {key}\n\n{value}\n")

Sampled parameter = w

       mean       std        5%       50%       95%
0  0.397923  0.489770  0.005756  0.207282  1.571548
1  0.840930  0.985975  0.002140  0.376907  3.210741
2  1.836145  2.281337  0.090406  0.941019  6.174343

Sampled parameter = b

       mean      std        5%       50%       95%
0  0.946605  0.06728  0.843839  0.945172  1.065237




3. Evaluate the regression fit on test data using MAE and MSE error metrics.

The most known metrics for comparing different regression models are the **Mean Absolute Error** (MAE)

$$\frac{1}{n}\sum_{i=1}^n |y_i - \hat{y}_i|$$

and the **Mean Squared Error** (MSE)

$$\frac{1}{n}\sum_{i=1}^n (y_i - \hat{y}_i)^2,$$

where $n$ is the number of observations, $y$ are the true values `y_test` and $\hat{y}$ are the predicted values `y_pred`.

In [61]:
# compute predictions using the inferred paramters
y_pred = torch.exp((inferred_w * X_test).sum(1)) + inferred_b

print("MAE =", torch.nn.L1Loss()(y_test, y_pred).item())
print("MSE =", torch.nn.MSELoss()(y_test, y_pred).item())

MAE = 562.3972778320312
MSE = 722826.5625


## Exercise 2

The Iris dataset contains petal and sepal length and width for three different types of Iris flowers: Setosa, Versicolour, and Virginica.





3. Evaluate your bayesian classifier on test data: compute the overall test accuracy and class-wise accuracy for the three different flower categories.

1. Import the Iris dataset from `sklearn`:
```
from sklearn import datasets
iris = datasets.load_iris()
```
and perform a train-test split on the data.

In [141]:
from sklearn import datasets
import matplotlib.pyplot as plt
iris = datasets.load_iris()

In [113]:
# dataset normalization and OneHotEncoding
iris_data = (iris.data - np.min(iris.data))/(np.max(iris.data)-np.min(iris.data))

# Each class will be 0 before vs rest will be 1 

# 0 vs rest dataset , just make 2 -> 1
zerovsrest = np.where((iris.target == 2),1,iris.target)
# 1 vs rest dataset , make 1 -> 0 and 0,2 -> 1
onevsrest = np.where((iris.target == 2),-1,iris.target)
onevsrest = np.where((onevsrest == 0),-1,onevsrest)
onevsrest = np.where((onevsrest == 1),0,onevsrest)
onevsrest = np.where((onevsrest == -1),1,onevsrest)

# 2 vs rest dataset, make 0,1 -> 1 and 2 -> 0
twovsrest = np.where((iris.target == 0),1,iris.target)
twovsrest = np.where((twovsrest == 2),0,twovsrest)

In [114]:
iris_data = torch.tensor(iris_data,dtype=torch.double)
iris_target = torch.tensor(iris.target,dtype=torch.double)
zerovsrest = torch.tensor(zerovsrest,dtype=torch.double)
onevsrest = torch.tensor(onevsrest,dtype=torch.double)
twovsrest = torch.tensor(twovsrest,dtype=torch.double)

2. Fit a multinomial bayesian logistic regression model on the four predictors petal length/width and sepal length/width. 

In [115]:
# model and guide functions
def log_reg_model(x, y):
    n_observations, n_predictors = x.shape
    
    w = pyro.sample("w", dist.Normal(torch.zeros(n_predictors), torch.ones(n_predictors)))
    b = pyro.sample("b", dist.Normal(0.,1.))
    
    # non-linearity
    yhat = torch.sigmoid((w*x).sum(dim=1) + b)
    
    with pyro.plate("data", n_observations):
        # sampling 0-1 labels from Bernoulli distribution
        y = pyro.sample("y", dist.Bernoulli(yhat), obs=y)
        
def log_reg_guide(x, y=None):
    
    n_observations, n_predictors = x.shape
    
    w_loc = pyro.param("w_loc", torch.rand(n_predictors))
    w_scale = pyro.param("w_scale", torch.rand(n_predictors), 
                         constraint=constraints.positive)
    w = pyro.sample("w", dist.Normal(w_loc, w_scale))
    
    b_loc = pyro.param("b_loc", torch.rand(1))
    b_scale = pyro.param("b_scale", torch.rand(1), 
                         constraint=constraints.positive)
    b = pyro.sample("b", dist.Normal(b_loc, b_scale))

In [116]:
def learn_infer_parameters(X_train ,y_train):
    # delete previously inferred params from pyro.param_store()
    pyro.clear_param_store()
    
    log_reg_svi = SVI(model=log_reg_model, guide=log_reg_guide, 
              optim=optim.ClippedAdam({'lr' : 0.0002}), 
              loss=Trace_ELBO()) 

    losses = []
    for step in range(10000):
        loss = log_reg_svi.step(X_train, y_train)/len(X_train)
        losses.append(loss)
        if step % 1000 == 0:
            print(f"Step {step} : loss = {loss}")
            
    w = pyro.get_param_store()["w_loc"]
    b = pyro.get_param_store()["b_loc"]
    
    return w,b

def predict_class(w,b,x):
    out = torch.sigmoid((w * x).sum(dim=1) + b)
    print(out)
    print(out>0.5)
    return (out > 0.5)

def return_prob(w,b,x):
    out = torch.sigmoid((w * x).sum(dim=1) + b)
    return out

In [117]:
fig, ax = plt.subplots(figsize=figsize)
ax.plot(losses)
ax.set_title("ELBO loss");

NameError: name 'figsize' is not defined

In [118]:
# split all the dataset , 0->0  vs 1,2 -> 1 ->
X_train_zero, X_test_zero, y_train_zero, y_test_zero = train_test_split(iris_data, zerovsrest, test_size=0.20, 
                                                    random_state=42)
w_zero , b_zero = learn_infer_parameters(X_train_zero, y_train_zero)

correct_predictions_0 = (predict_class(w_zero, b_zero, X_test_zero) == y_test_zero).sum().item()
out_zero = return_prob(w_zero, b_zero, X_test_zero)
print(f"test accuracy = {correct_predictions_0/len(X_test_zero)*100:.2f}%")

Step 0 : loss = 0.755019464279618
Step 1000 : loss = 0.7127734735863194
Step 2000 : loss = 0.6822780439594306
Step 3000 : loss = 0.7716861839703602
Step 4000 : loss = 0.5577056783150035
Step 5000 : loss = 0.6744040640989016
Step 6000 : loss = 0.5955440425212685
Step 7000 : loss = 0.5436122392058109
Step 8000 : loss = 0.40504829698803096
Step 9000 : loss = 0.5070481985581807
tensor([0.7545, 0.5470, 0.8737, 0.7596, 0.7715, 0.5457, 0.7099, 0.8179, 0.7670,
        0.7209, 0.8034, 0.5216, 0.5217, 0.5264, 0.5320, 0.7699, 0.8346, 0.7165,
        0.7492, 0.8296, 0.5354, 0.7877, 0.5480, 0.8259, 0.8450, 0.8209, 0.8243,
        0.8409, 0.5345, 0.5374], dtype=torch.float64,
       grad_fn=<SigmoidBackward>)
tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True])
test accuracy = 66.67%


In [120]:
# split all the dataset, 1 -> 0 , 0,2 ->1
X_train_one, X_test_one, y_train_one, y_test_one = train_test_split(iris_data, onevsrest, test_size=0.20, 
                                                    random_state=42)

w_one , b_one = learn_infer_parameters(X_train_one, y_train_one)
correct_predictions_1 = (predict_class(w_one, b_one, X_test_one) == y_test_one).sum().item()
out_one = return_prob(w_one, b_one, X_test_one)
print(f"test accuracy = {correct_predictions_1/len(X_test_one)*100:.2f}%")

Step 0 : loss = 0.7622346981497066
Step 1000 : loss = 0.7583319368874285
Step 2000 : loss = 0.8202136347158617
Step 3000 : loss = 0.6931911362867472
Step 4000 : loss = 0.7069921968572507
Step 5000 : loss = 0.7164076395192318
Step 6000 : loss = 0.6647996456930353
Step 7000 : loss = 0.7017418486098015
Step 8000 : loss = 0.6846783682758677
Step 9000 : loss = 0.6635610847387302
tensor([0.6469, 0.6742, 0.6491, 0.6457, 0.6528, 0.6616, 0.6436, 0.6529, 0.6320,
        0.6419, 0.6532, 0.6487, 0.6666, 0.6520, 0.6680, 0.6569, 0.6466, 0.6360,
        0.6418, 0.6411, 0.6511, 0.6462, 0.6572, 0.6419, 0.6804, 0.6484, 0.6409,
        0.6534, 0.6470, 0.6500], dtype=torch.float64,
       grad_fn=<SigmoidBackward>)
tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True])
test accuracy = 70.00%


In [121]:
# split all the dataset 2->0 , 0,1 -> 1
X_train_two, X_test_two, y_train_two, y_test_two = train_test_split(iris_data, twovsrest, test_size=0.20, 
                                                    random_state=42)

w_two , b_two = learn_infer_parameters(X_train_two, y_train_two)
correct_predictions_2 = (predict_class(w_two, b_two, X_test_two) == y_test_two).sum().item()
out_two = return_prob(w_two, b_two, X_test_two)
print(f"test accuracy = {correct_predictions_2/len(X_test_two)*100:.2f}%")

Step 0 : loss = 0.7802328453812867
Step 1000 : loss = 0.6617454357905336
Step 2000 : loss = 0.6906303834054249
Step 3000 : loss = 0.7056654246237097
Step 4000 : loss = 0.6625433119263082
Step 5000 : loss = 0.6278532156133321
Step 6000 : loss = 0.6447132386968544
Step 7000 : loss = 0.776075068137491
Step 8000 : loss = 0.5476864286437073
Step 9000 : loss = 0.5873024768809179
tensor([0.6151, 0.7435, 0.5197, 0.6141, 0.6110, 0.7348, 0.6433, 0.5840, 0.5953,
        0.6334, 0.5933, 0.7318, 0.7473, 0.7323, 0.7448, 0.6187, 0.5600, 0.6294,
        0.6152, 0.5594, 0.7286, 0.5967, 0.7301, 0.5626, 0.5817, 0.5766, 0.5591,
        0.5619, 0.7268, 0.7268], dtype=torch.float64,
       grad_fn=<SigmoidBackward>)
tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True])
test accuracy = 63.33%


In [122]:
X_train, X_test, y_train, y_test = train_test_split(iris_data, iris_target, test_size=0.20, 
                                                    random_state=42)
y_test,y_test_zero, y_test_one ,y_test_two
        

(tensor([1., 0., 2., 1., 1., 0., 1., 2., 1., 1., 2., 0., 0., 0., 0., 1., 2., 1.,
         1., 2., 0., 2., 0., 2., 2., 2., 2., 2., 0., 0.], dtype=torch.float64),
 tensor([1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1.,
         1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0.], dtype=torch.float64),
 tensor([0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0.,
         0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=torch.float64),
 tensor([1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1.,
         1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.], dtype=torch.float64))

In [132]:
out_final = torch.stack((out_zero,out_one,out_two),dim=1)
out_final


tensor([[0.7545, 0.6469, 0.6151],
        [0.5470, 0.6742, 0.7435],
        [0.8737, 0.6491, 0.5197],
        [0.7596, 0.6457, 0.6141],
        [0.7715, 0.6528, 0.6110],
        [0.5457, 0.6616, 0.7348],
        [0.7099, 0.6436, 0.6433],
        [0.8179, 0.6529, 0.5840],
        [0.7670, 0.6320, 0.5953],
        [0.7209, 0.6419, 0.6334],
        [0.8034, 0.6532, 0.5933],
        [0.5216, 0.6487, 0.7318],
        [0.5217, 0.6666, 0.7473],
        [0.5264, 0.6520, 0.7323],
        [0.5320, 0.6680, 0.7448],
        [0.7699, 0.6569, 0.6187],
        [0.8346, 0.6466, 0.5600],
        [0.7165, 0.6360, 0.6294],
        [0.7492, 0.6418, 0.6152],
        [0.8296, 0.6411, 0.5594],
        [0.5354, 0.6511, 0.7286],
        [0.7877, 0.6462, 0.5967],
        [0.5480, 0.6572, 0.7301],
        [0.8259, 0.6419, 0.5626],
        [0.8450, 0.6804, 0.5817],
        [0.8209, 0.6484, 0.5766],
        [0.8243, 0.6409, 0.5591],
        [0.8409, 0.6534, 0.5619],
        [0.5345, 0.6470, 0.7268],
        [0.537

In [138]:
torch.argmin(out_final,dim=1) == y_test

tensor([False,  True,  True, False, False,  True, False,  True, False, False,
         True,  True,  True,  True,  True, False,  True, False, False,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True])

In [140]:
torch.argmin(out_final,dim=1)

tensor([2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2,
        2, 2, 2, 2, 0, 0], grad_fn=<NotImplemented>)

In [139]:
correct_predictions_final = (torch.argmin(out_final,dim=1) == y_test).sum().item()
print(f"test accuracy = {correct_predictions_final/len(X_test)*100:.2f}%")

test accuracy = 70.00%


Our Bayesian logistic regression function returns a probability score between 0 and 1 on each input point $x$, which is the output of the sigmoid function. 

In order to map this value to a discrete class, `predict_class()` uses a **threshold value** of 0.5 to decide whether $x$ belongs to class $0$ or $1$. 


Finally, we can compute the **test accuracy** of our model, which is the percentage of correct predictions on test data.

In [79]:
correct_predictions = (predict_class(X_test) == y_test).sum().item()

print(f"test accuracy = {correct_predictions/len(X_test)*100:.2f}%")

test accuracy = 0.00%
